In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model, Sequential

import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

import random
random.seed(1228)

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

%matplotlib inline

Using TensorFlow backend.


### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

#### Reading the data:

In [4]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [14]:
columns = ['ignore', 'words', 'ignore', 'pos', 'chunk']
train_reader = ConllCorpusReader(root = '.', fileids = 'unamb_sent_14_6.conllu', columntypes = columns)


sents = train_reader.iob_sents()
train_reader.iob_sents()

[[('«', 'PUNCT', '_'), ('Школа', 'NOUN', '_'), ('злословия', 'NOUN', '_'), ('»', 'PUNCT', '_'), ('учит', 'VERB', '_'), ('прикусить', 'VERB', '_'), ('язык', 'NOUN', '_')], [('Сохранится', 'VERB', '_'), ('ли', 'PART', '_'), ('градус', 'NOUN', '_'), ('дискуссии', 'NOUN', '_'), ('в', 'ADP', '_'), ('новом', 'ADJ', '_'), ('сезоне', 'NOUN', '_'), ('?', 'PUNCT', '_')], ...]

#### Считаем количество предложений и разных тэгов

In [28]:
len(sents) # Предложений в корпусе 38508

38508

In [21]:
#вытащим все тэги
pos = [w[1] for sent in sents for w in sent]
pos[:15]

['PUNCT',
 'NOUN',
 'NOUN',
 'PUNCT',
 'VERB',
 'VERB',
 'NOUN',
 'VERB',
 'PART',
 'NOUN',
 'NOUN',
 'ADP',
 'ADJ',
 'NOUN',
 'PUNCT']

In [22]:
# вот столько у нас разных тэгов
len(set(pos))

14

In [25]:
# а теперь посмотрим на частотное распределение тэгов 
from nltk import FreqDist
fd_pos = FreqDist(pos)

In [26]:
fd_pos

FreqDist({'ADJ': 47487,
          'ADP': 42835,
          'ADV': 13079,
          'CONJ': 21942,
          'DET': 12689,
          'INTJ': 452,
          'NOUN': 121793,
          'NUM': 10173,
          'PART': 8923,
          'PRON': 9067,
          'PROPN': 14889,
          'PUNCT': 91323,
          'VERB': 41538,
          'X': 21393})

Как мы видим, самые частые -- прилагательные и предлоги (о_О)

#### Сформируем тестовую и тренировочную выборки для тэгов

Итак, вектор POS у нас есть: `pos`. Теперь сделаем вектор признаков. Для удобства мы решили убрать информацию о границах предложений.

In [44]:
words = [word[0] for sent in sents for word in sent]
words[:20]

['«',
 'Школа',
 'злословия',
 '»',
 'учит',
 'прикусить',
 'язык',
 'Сохранится',
 'ли',
 'градус',
 'дискуссии',
 'в',
 'новом',
 'сезоне',
 '?',
 'Великолепная',
 '«',
 'Школа',
 'злословия',
 '»']

Извлечение признаков:

In [46]:
embeddingsPath = './wiki.ru.vec'

In [54]:
def extract_embeddings(path):
    embeddings = {}
    with open(embeddingsPath) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            embeddings[word] = [float(num) for num in vec.split()]
    return embeddings

embeddings = extract_embeddings(embeddingsPath)

In [55]:
embeddings['школа']

[-0.32315,
 0.91456,
 0.13797,
 0.61075,
 -0.28406,
 -0.47918,
 -0.27341,
 0.17947,
 0.54726,
 -0.47914,
 -0.20418,
 0.12833,
 0.1399,
 0.26005,
 0.53394,
 -0.0071751,
 -0.7064,
 -0.72035,
 0.52635,
 0.26623,
 -0.2057,
 -0.19932,
 -0.22531,
 -0.40999,
 -0.23029,
 -0.3556,
 0.052551,
 -0.17581,
 0.37426,
 0.6961,
 -0.10287,
 0.096703,
 0.1357,
 -0.050066,
 -0.080225,
 0.63446,
 0.52929,
 -0.68261,
 0.14481,
 0.38492,
 -0.029405,
 -0.051461,
 0.11573,
 0.11116,
 -0.10452,
 -0.27637,
 0.023329,
 -0.34237,
 0.081127,
 -0.55629,
 0.084345,
 -0.2759,
 0.088321,
 0.16879,
 0.49636,
 -0.039692,
 0.4441,
 0.43682,
 0.49919,
 -0.47123,
 0.26761,
 -0.36185,
 -0.22721,
 0.32502,
 0.11182,
 -0.38456,
 0.0081887,
 0.73195,
 0.50026,
 -0.25915,
 -0.27511,
 0.054941,
 -0.31678,
 0.024292,
 -0.23829,
 0.11533,
 -0.29501,
 -0.089492,
 0.11684,
 0.021585,
 -0.22037,
 0.011673,
 -0.29228,
 0.2409,
 -0.05682,
 0.099456,
 -0.016484,
 0.24158,
 0.35266,
 0.42744,
 -0.33701,
 0.17153,
 0.19474,
 0.33683,
 -0.

In [56]:
len(embeddings['школа'])

300

In [57]:
# создаём вектора для начала и конца
BOF = [0] * 300
EOF = [0] * 300

In [58]:
# k == 2
def extract_featues(tokens):
    features = []
    for i, word in enumerate(tokens):
        cur_word = embeddings[word]
        prev_word = embedding[tokens[i - 1]]
        next_word = embedding[tokens[i + 1]]
        features.append(cur_word + prev_word + next_word)
    return features

In [ ]:
len(featueres)

In [45]:
boundary = (len(pos) // 4) * 3
y_train = pos[:boundary]
y_test = pos[boundary:]

print(len(y_train))
print(len(y_test))

343185
114398


### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

я бы поставила семь, конечно, но можно и три пока а это для последнего задания

In [ ]:
TEXT_LENGTH = 1800
VOCABULARY_SIZE = 250000
EMBEDDING_DIM = 300
DIMS = 250
MAX_FEATURES = 5000
batch_size = 32

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 10

In [ ]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_corpus.body)
tokenizer

In [42]:
sequences = tokenizer.texts_to_sequences(train_s.body)
X_train = tokenizer.sequences_to_matrix(sequences, mode='count')
sequences = tokenizer.texts_to_sequences(test_corpus.body)
X_test = tokenizer.sequences_to_matrix(sequences, mode='count')

NameError: name 'train_s' is not defined

Посмотреть какая именно архитектура сенны

In [ ]:
model = Sequential()
model.add(Embedding(VOCABULARY_SIZE, EMBEDDING_DIM, input_length=TEXT_LENGTH))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))
model.add(Dropout(0.5))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)